In [275]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [276]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', required=True, help='cifar10 | lsun | imagenet | folder | lfw | fake')
parser.add_argument('--dataroot', required=True, help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imageSize', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=100, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64)
parser.add_argument('--ndf', type=int, default=64)
parser.add_argument('--niter', type=int, default=25, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--outf', default='.', help='folder to output images and model checkpoints')
parser.add_argument('--manualSeed', type=int, help='manual seed')

_StoreAction(option_strings=['--manualSeed'], dest='manualSeed', nargs=None, const=None, default=None, type=<type 'int'>, choices=None, help='manual seed', metavar=None)

In [277]:
args = {}

In [278]:
args['batch_size']=100
args['test_batch_size'] =1000

In [279]:
args['manualSeed'] = 5
print("Random Seed: ", args['manualSeed'])

Random Seed:  5


In [280]:
random.seed(args['manualSeed'])
torch.manual_seed(args['manualSeed'])

In [281]:
kwargs = {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

In [282]:
def _calculate_fan_in_and_fan_out(tensor):
    dimensions = tensor.ndimension()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with less than 2 dimensions")

    if dimensions == 2:  # Linear
        fan_in = tensor.size(1)
        fan_out = tensor.size(0)
    else:
        num_input_fmaps = tensor.size(1)
        num_output_fmaps = tensor.size(0)
        receptive_field_size = 1
        if tensor.dim() > 2:
            receptive_field_size = tensor[0][0].numel()
        fan_in = num_input_fmaps * receptive_field_size
        fan_out = num_output_fmaps * receptive_field_size

    return fan_in, fan_out

In [283]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
       # self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
      #  x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [306]:
neural_networks = {}

In [307]:
n_networks = 10
for i in range(n_networks):
    neural_networks[i] = Net()

In [308]:
import math

In [309]:
def weights_init(m, xavier=True, std_given=10., random_seed=1):
    classname = m.__class__.__name__
    random.seed(random_seed)
    torch.manual_seed(random_seed)
    print (classname)
    if classname.find('Conv') != -1:
       # m.weight.data.normal_(0.0, 0.02)
        
        if xavier:
            z = _calculate_fan_in_and_fan_out(m.weight.data)
            std = math.sqrt(2.0 / (z[0] + z[1]))
            m.weight.data.uniform_(-std, std)
        else:
            m.weight.data.uniform_(-std_given, std_given)
    if classname.find('Linear') != -1:
        if xavier:
            z = _calculate_fan_in_and_fan_out(m.weight.data)
            std = math.sqrt(2.0 / (z[0] + z[1]))
            m.weight.data.uniform_(-std, std)
            
            m.bias.data.fill_(0.0)
        else:
            m.weight.data.uniform_(-std_given, std_given)
            m.bias.data.uniform_(-std_given, std_given)



In [310]:
for i in range(1, n_networks - 1):
    magnitudes = 10 ** (- i + 4)
    g = lambda x: weights_init(x, xavier=False, std_given=magnitudes, random_seed=i)
    neural_networks[i].apply(g)

Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net


In [322]:
z=list(neural_networks[9].parameters())
z[0]

Parameter containing:
(0 ,0 ,.,.) = 
  0.0549  0.1943 -0.1287  0.0482 -0.1890
  0.1021  0.1862 -0.1877 -0.1574 -0.1975
  0.0497  0.0365  0.0583 -0.1731 -0.1708
  0.1353  0.1250 -0.1063  0.1920  0.0898
  0.0657 -0.1477  0.1153 -0.1384 -0.1394

(1 ,0 ,.,.) = 
  0.0384  0.0115  0.1107 -0.1991 -0.0923
  0.1488 -0.0196 -0.0450 -0.0378  0.1518
 -0.0167 -0.0258 -0.0344 -0.0179 -0.1215
  0.1152 -0.1092  0.0906  0.1200 -0.1950
  0.0801 -0.1447 -0.1155  0.1131  0.0719

(2 ,0 ,.,.) = 
 -0.1394  0.0344  0.1518  0.1342  0.0521
  0.1644  0.1650  0.0347 -0.1053 -0.1411
 -0.0823  0.1888 -0.1808 -0.1411 -0.1516
 -0.1491 -0.1189  0.1800  0.0182  0.0174
  0.1368 -0.0452 -0.0520 -0.1024  0.0933

(3 ,0 ,.,.) = 
  0.1441  0.1597  0.0736 -0.1854  0.1572
  0.0377 -0.0340 -0.0472 -0.0161  0.1869
 -0.0773 -0.0068 -0.0037  0.1552  0.0608
  0.1361  0.0094  0.1046  0.0421  0.1084
 -0.0929 -0.0618 -0.0526 -0.0997  0.0586

(4 ,0 ,.,.) = 
 -0.0483  0.0806  0.0401  0.0348 -0.0240
  0.0256  0.1308 -0.1005 -0.0335  0.02

In [323]:
args['lr'] = 0.01
args['momentum'] = 0.0

In [324]:
args['cuda'] = False

In [325]:
optimizers = {}

for i in neural_networks:
    optimizers[i] = optim.SGD(neural_networks[i].parameters(), lr=args['lr'], momentum=args['momentum'])

In [326]:
def train(epoch, model, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#         if batch_idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.data[0]))

    model.eval()
    train_loss = 0
    correct = 0
    for data, target in train_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    return 100. * correct / len(train_loader.dataset)

In [327]:
results = {}
for i in range(n_networks):
    results[i] = []

In [ ]:
for epoch in range(1, 50 + 1):
    for i in range(n_networks):
        res = train(epoch, neural_networks[i], optimizers[i])
        results[i].append(res)